In [5]:
import pandas as pd
read_file = pd.read_csv ('train')
read_file.to_csv ('train.csv', index=None)
read_file = pd.read_csv ('test')
read_file.to_csv ('test.csv', index=None)

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt 
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from deepctr.models import DCN #使用Deep & Cross
from deepctr.inputs import SparseFeat,get_feature_names
import pickle


##==================== 设置文件路径File-Path (fp) ====================##
# file_path = '/home/admin/avazu/'
# fp_train_f = file_path + "sub_train_f.csv" #使用小样本进行训练
fp_train_f =  "train.csv"
##==================== DCN 训练 ====================##
data = pd.read_csv(fp_train_f, dtype={'id':str}, index_col=None)
print('data loaded')

#数据加载
sparse_features = ['C1', 'banner_pos', 'site_domain', 'site_id','site_category','app_id','app_category', 'device_type', 'device_conn_type','C14', 'C15','C16']
target = ['click']

# 对特征标签进行编码
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature])
# 计算每个特征中的 不同特征值的个数
fixlen_feature_columns = [SparseFeat(feature, data[feature].nunique()) for feature in sparse_features]
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)
print(fixlen_feature_columns)
print(feature_names)

# 将数据集切分成训练集和测试集
train, test = train_test_split(data, test_size=0.2)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

# 使用DCN进行训练
#model = DCN(linear_feature_columns, dnn_feature_columns, task='regression')
model = DCN(linear_feature_columns, dnn_feature_columns, task='binary')
model.compile("adam", "binary_crossentropy", metrics=['binary_crossentropy'], )
history = model.fit(train_model_input, train[target].values, batch_size=256, epochs=1, verbose=True, validation_split=0.2, )
# 使用DCN进行预测
pred_ans = model.predict(test_model_input, batch_size=256)
# 输出RMSE或MSE
mse = round(mean_squared_error(test[target].values, pred_ans), 4)
rmse = mse ** 0.5
print("test RMSE", rmse)

# 输出LogLoss
from sklearn.metrics import log_loss
score = log_loss(test[target].values, pred_ans)
print("LogLoss", score)

DeepCTR version 0.7.4 detected. Your version is 0.7.3.
Use `pip install -U deepctr` to upgrade.Changelog: https://github.com/shenweichen/DeepCTR/releases/tag/v0.7.4


data loaded
[SparseFeat(name='C1', vocabulary_size=7, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='C1', group_name='default_group'), SparseFeat(name='banner_pos', vocabulary_size=7, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='banner_pos', group_name='default_group'), SparseFeat(name='site_domain', vocabulary_size=7745, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='site_domain', group_name='default_group'), SparseFeat(name='site_id', vocabulary_size=4737, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='site_id', group_name='default_group'), SparseFeat(name='site_category', vocabulary_size=26, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='site_category', group_name='default_group'), SparseFeat(name='app_id', vocabulary_size=8552, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='app_id', group_name='default_group'), SparseFeat(name='app_category', vocabulary_size=36, embedding_dim=4, 

/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


25874538/25874538 [==============================] - 382s 15us/sample - loss: 0.4008 - binary_crossentropy: 0.3985 - val_loss: 0.4002 - val_binary_crossentropy: 0.3977
test RMSE 0.3531288716601915
LogLoss 0.39762907702265143
